### Tensorboard and data setup

In [2]:
# Data, clone git
!git clone https://ashencz:@github.com/AshenCZ/deep-learning.git

# Change dir to the project
# TODO: Change directory of project
%cd ./deep-learning/3d_recce
!ls

fatal: destination path 'deep-learning' already exists and is not an empty directory.
/content/deep-learning/3d_recce
logs  modelnet20-test.npz  modelnet20-train.npz


In [36]:
# Tensorboard

!git clone https://github.com/mixuala/colab_utils.git

import os
import colab_utils.tboard

# set paths
ROOT = %pwd
LOG_DIR = os.path.join(ROOT, 'logs')
print(ROOT, LOG_DIR)

# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
colab_utils.tboard.launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

fatal: destination path 'colab_utils' already exists and is not an empty directory.
/content/deep-learning/3d_recce /content/deep-learning/3d_recce/logs
ngrok installed
status: tensorboard=True, ngrok=False
tensorboard url= https://6b587b83.ngrok.io


'https://6b587b83.ngrok.io'

In [0]:
# Find out how much GPU memory you have left

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

## Code

In [0]:
#!/usr/bin/env python3
import numpy as np
import tensorflow as tf

class Args(object):
  pass

In [0]:
class Dataset:
    def __init__(self, filename, shuffle_batches = True):
        data = np.load(filename)
        self._voxels = data["voxels"]
        self._labels = data["labels"] if "labels" in data else None

        self._shuffle_batches = shuffle_batches
        self._new_permutation()

    def _new_permutation(self):
        if self._shuffle_batches:
            self._permutation = np.random.permutation(len(self._voxels))
        else:
            self._permutation = np.arange(len(self._voxels))

    def split(self, ratio):
        split = int(len(self._voxels) * ratio)

        first, second = Dataset.__new__(Dataset), Dataset.__new__(Dataset)
        first._voxels, second._voxels = self._voxels[:split], self._voxels[split:]
        if self._labels is not None:
            first._labels, second._labels = self._labels[:split], self._labels[split:]
        else:
            first._labels, second._labels = None, None

        for dataset in [first, second]:
            dataset._shuffle_batches = self._shuffle_batches
            dataset._new_permutation()

        return first, second

    @property
    def voxels(self):
        return self._voxels

    @property
    def labels(self):
        return self._labels

    def next_batch(self, batch_size):
        batch_size = min(batch_size, len(self._permutation))
        batch_perm, self._permutation = self._permutation[:batch_size], self._permutation[batch_size:]
        return self._voxels[batch_perm], self._labels[batch_perm] if self._labels is not None else None

    def epoch_finished(self):
        if len(self._permutation) == 0:
            self._new_permutation()
            return True
        return False



### Network

In [0]:
class Network:
    LABELS = 10

    def __init__(self, threads, seed=42):
        # Create an empty graph and a session
        graph = tf.Graph()
        graph.seed = seed
        self.session = tf.Session(graph = graph, config=tf.ConfigProto(inter_op_parallelism_threads=threads,
                                                                       intra_op_parallelism_threads=threads))

    def parse_architecture(self, arch):
        layers = arch.split(",")
        result = []
        for l in layers:
            params = l.split("-")
            parsed_args = [int(x) if x.isdigit() else x for x in params[1:]]
            result.append((params[0], parsed_args))
        print(result)
        return result

    def construct(self, args):
        with self.session.graph.as_default():
            # Inputs
            self.voxels = tf.placeholder(
                tf.float32, [None, args.modelnet_dim, args.modelnet_dim, args.modelnet_dim, 1], name="voxels")
            self.labels = tf.placeholder(tf.int64, [None], name="labels")
            self.is_training = tf.placeholder(tf.bool, [], name="is_training")

            global_step = tf.train.create_global_step()
            last_layer = self.voxels

            archi = self.parse_architecture(args.cnn)
            for (name, params) in archi:
                if name == "F": last_layer = tf.layers.flatten(last_layer)
                elif name == "R": last_layer = tf.layers.dense(last_layer, units=params[0], activation=tf.nn.relu)
                elif name == "D": last_layer = tf.layers.dropout(last_layer, rate=0.5, training=self.is_training)
                elif name == "C": last_layer = tf.layers.conv3d(last_layer, filters=params[0], kernel_size=params[1], strides=params[2], padding=params[3], activation=tf.nn.relu)
                elif name == "M": last_layer = tf.layers.max_pooling3d(last_layer, pool_size=params[0], strides=params[1])
                elif name == "CB":
                    last_layer = tf.layers.conv3d(last_layer, filters=params[0], kernel_size=params[1], strides=params[2], padding=params[3], activation=None, use_bias=False)
                    last_layer = tf.layers.batch_normalization(last_layer, training=self.is_training)
                    last_layer = tf.nn.relu(last_layer)
                else:
                    raise Exception("What.")

            pred_layer = tf.layers.dense(last_layer, units=10, activation=tf.nn.relu)            
            self.predictions = tf.argmax(pred_layer, axis=1)  # shape [None] and type tf.int64

            # Training
            learning_rate = tf.train.exponential_decay(args.learning_rate, global_step, batches_per_epoch, (args.learning_rate_final / args.learning_rate) ** (1/(args.epochs-1)), True)
            loss = tf.losses.sparse_softmax_cross_entropy(self.labels, pred_layer, scope="loss_pred")

            # Update normalizations
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                self.training = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step, name="training")
            
            # Summaries
            self.accuracy = tf.reduce_mean(tf.cast(tf.equal(self.labels, self.predictions), tf.float32))
            summary_writer = tf.contrib.summary.create_file_writer(args.logdir, flush_millis=10 * 1000)
            self.summaries = {}
            with summary_writer.as_default(), tf.contrib.summary.record_summaries_every_n_global_steps(8):
                self.summaries["train"] = [tf.contrib.summary.scalar("train/loss", loss),
                                           tf.contrib.summary.scalar("train/accuracy", self.accuracy)]
            with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():
                for dataset in ["dev", "test"]:
                    self.summaries[dataset] = [tf.contrib.summary.scalar(dataset + "/loss", loss),
                                               tf.contrib.summary.scalar(dataset + "/accuracy", self.accuracy)]

            # Initialize variables
            self.session.run(tf.global_variables_initializer())
            with summary_writer.as_default():
                tf.contrib.summary.initialize(session=self.session, graph=self.session.graph)

    def train(self, voxels, labels):
        self.session.run([self.training, self.summaries["train"]], {self.voxels: voxels, self.labels: labels, self.is_training: True})

    def evaluate(self, dataset, voxels, labels):
        accuracy, _ = self.session.run([self.accuracy, self.summaries[dataset]], {self.voxels: voxels, self.labels: labels, self.is_training: False})
        return accuracy

    def predict(self, voxels):
        return self.session.run(self.predictions, {self.voxels: voxels, self.is_training: False})


### Training

In [37]:
import datetime
import os
import re

# Fix random seed
np.random.seed(42)

# Parse arguments
args = Args()
args.epochs = 80
args.threads = 4    
args.batch_size = 128
args.learning_rate = 0.001
args.learning_rate_final = 0.0001
args.modelnet_dim = 20
args.cnn = "CB-32-3-1-same,CB-32-3-1-same,M-3-2,CB-64-3-1-same,CB-64-3-1-same,M-3-2,CB-128-3-1-same,M-3-2,F,R-4096,D"
args.train_split = 0.9   

# Create logdir name
args.logdir = "logs/{}-{}-{}".format(
    os.path.basename("./"),
    datetime.datetime.now().strftime("%Y-%m-%d_%H%M%S"),
    ",".join(("{}={}".format(re.sub("(.)[^_]*_?", r"\1", key), value) for key, value in sorted(vars(args).items())))
)
if not os.path.exists("logs"): os.mkdir("logs") # TF 1.6 will do this by itself

# Load the data
train, dev = Dataset("modelnet{}-train.npz".format(args.modelnet_dim)).split(args.train_split)
test = Dataset("modelnet{}-test.npz".format(args.modelnet_dim), shuffle_batches=False)

# Construct the network
print("Lens:",len(train.labels), len(dev.labels))
batches_per_epoch = len(train.labels) // args.batch_size
network = Network(threads=args.threads)
network.construct(args)

# Train
for i in range(args.epochs):
    print("Training epoch",i)
    while not train.epoch_finished():
        voxels, labels = train.next_batch(args.batch_size)
        network.train(voxels, labels)

    acc = network.evaluate("dev", dev.voxels, dev.labels)
    print("Dev at",i,":",acc)

# # Predict test data
#with open("{}/3d_recognition_test.txt".format(args.logdir), "w") as test_file:
#    while not test.epoch_finished():
#        voxels, _ = test.next_batch(args.batch_size)
#        labels = network.predict(voxels)

        #for label in labels:
         #   print(label, file=test_file)


Lens: 3591 400
[('CB', [32, 3, 1, 'same']), ('CB', [32, 3, 1, 'same']), ('M', [3, 2]), ('CB', [64, 3, 1, 'same']), ('CB', [64, 3, 1, 'same']), ('M', [3, 2]), ('CB', [128, 3, 1, 'same']), ('M', [3, 2]), ('F', []), ('R', [4096]), ('D', [])]
Training epoch 0
Dev at 0 : 0.17
Training epoch 1
Dev at 1 : 0.3775
Training epoch 2
Dev at 2 : 0.1
Training epoch 3
Dev at 3 : 0.1
Training epoch 4
Dev at 4 : 0.1
Training epoch 5
Dev at 5 : 0.1
Training epoch 6
Dev at 6 : 0.1
Training epoch 7
Dev at 7 : 0.1
Training epoch 8
Dev at 8 : 0.1
Training epoch 9
Dev at 9 : 0.1
Training epoch 10
Dev at 10 : 0.4075
Training epoch 11
Dev at 11 : 0.2875
Training epoch 12
Dev at 12 : 0.27
Training epoch 13
Dev at 13 : 0.3975
Training epoch 14
Dev at 14 : 0.4075
Training epoch 15
Dev at 15 : 0.475
Training epoch 16
Dev at 16 : 0.4825
Training epoch 17
Dev at 17 : 0.5125
Training epoch 18
Dev at 18 : 0.5225
Training epoch 19
Dev at 19 : 0.525
Training epoch 20
Dev at 20 : 0.5475
Training epoch 21
Dev at 21 : 0.54

KeyboardInterrupt: ignored

### Test set evaluation

In [29]:
# Predict test data
print("3d_recognition_test.txt")
with open("3d_recognition_test.txt", "w") as test_file:
    while not test.epoch_finished():
        voxels, _ = test.next_batch(args.batch_size)
        labels = network.predict(voxels)

        for label in labels:
            print(label, file=test_file)

3d_recognition_test.txt


In [31]:
# Download the ouput data
!ls
files.download('3d_recognition_test.txt')

3d_recognition_test.txt  logs		      modelnet20-train.npz
colab_utils		 modelnet20-test.npz  ngrok


## The rest

In [0]:
# Blank for tests
# !kill -9 -1

#file = "/events.out.tfevents.1522837908.bac2a7f9746a.v2"

#%cd logs/
#!ls
#!mv events.out.tfevents.1522837908.bac2a7f9746a.v2 a.txt

from google.colab import files
files.download("a.txt")

In [42]:
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"
!git commit -m "Bamboozling logs."
!git push https://ashencz:@github.com/AshenCZ/deep-learning.git/

[master 22f9a03] Bamboozling logs.
 14 files changed, 908 insertions(+)
 create mode 100644 3d_recce/logs/-2018-04-09_210645-bs=256,c=C-8-3-1-same,M-2-2,C-16-3-1-same,M-2-2,F,R-512,D,e=20,lr=0.001,lrf=0.0002,md=20,t=4,ts=0.75/events.out.tfevents.1523308007.c0203b8ed32d.v2
 create mode 100644 3d_recce/logs/-2018-04-09_210759-bs=256,c=C-8-3-1-same,C-8-3-1-same,M-3-2,C-16-3-1-same,C-16-3-1-same,M-2-2,F,R-512,D,e=20,lr=0.001,lrf=0.0002,md=20,t=4,ts=0.75/events.out.tfevents.1523308080.c0203b8ed32d.v2
 create mode 100644 3d_recce/logs/-2018-04-09_210918-bs=256,c=CB-8-3-1-same,CB-8-3-1-same,M-2-2,CB-16-3-1-same,CB-16-3-1-same,M-2-2,F,R-512,D,e=20,lr=0.001,lrf=0.0002,md=20,t=4,ts=0.75/events.out.tfevents.1523308159.c0203b8ed32d.v2
 create mode 100644 3d_recce/logs/-2018-04-09_211147-bs=256,c=CB-8-3-1-same,CB-8-3-1-same,M-2-2,CB-16-3-1-same,CB-16-3-1-same,M-2-2,F,R-512,D,e=50,lr=0.001,lrf=0.0002,md=20,t=4,ts=0.75/events.out.tfevents.1523308309.c0203b8ed32d.v2
 create mode 100644 3d_recce/logs/-

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving events.out.tfevents.1522837908.bac2a7f9746a.v2 to events.out.tfevents.1522837908.bac2a7f9746a.v2
User uploaded file "events.out.tfevents.1522837908.bac2a7f9746a.v2" with length 873750 bytes


In [0]:
print(acc)

0.9286
